In [1]:
# Imports
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

print(tf.__version__)

2.5.0


# Saving and loading models

***
It's possible to manually save the weights of a model, for example, after a training run has completed. 

There are two different formats available for saving and loading files: a native TensorFlow format as well as the HDF5 format used by Keras. 


***


## Preparing Dataset 

***



In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Define the Model

In [3]:
# Define a simple sequential model
def create_model():
  model = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Saving only Weights using the Modelcheckpoint Callback



In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_dir = '/content/modelcheckpoints'

# deleting any directory if there is any
!rm -r modelcheckpoints

cp_callback = ModelCheckpoint(
    filepath = checkpoint_dir,
    monitor = 'val_loss',
    verbose = 2,
    save_best_only = False,
    save_weights_only = True,
    mode = 'auto',
    save_freq = 'epoch'
)

rm: cannot remove 'modelcheckpoints': No such file or directory


Now use the above instance `cp_callback` int the `model.fit` method.

In [5]:
history = model.fit(
    x = train_images, y = train_labels,
    batch_size = 64, epochs = 5, verbose = 0, callbacks = [cp_callback],
    validation_data = (test_images, test_labels)
)


Epoch 00001: saving model to /content/modelcheckpoints

Epoch 00002: saving model to /content/modelcheckpoints

Epoch 00003: saving model to /content/modelcheckpoints

Epoch 00004: saving model to /content/modelcheckpoints

Epoch 00005: saving model to /content/modelcheckpoints


Let's check original model's loss and with loaded wieghts loss.

In [6]:
loss, acc = model.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Original Model is {:.4f}.'.format(loss))

# Build a new model
model_1 = create_model()
model_1.load_weights(filepath=checkpoint_dir)
loss, acc = model_1.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Loaded Weights is {:.4f}.'.format(loss))

Test Loss with Original Model is 0.4444.
Test Loss with Loaded Weights is 0.4444.


***



## Saving and loading models in h5 Formate

There's also an alternative formats that we can use to save the model weights. The formats used by Keras is the HDF5 format. These file formats often use the extension .h5. 

If we give the file path argument, the h5 extension, as shown below, then the model will save the weights in the Keras HDF5 format. 

```
checkpoint = ModelCheckpoint('my_model.h5', save_weights_only = True)
```

After running the training, we will see that the model weights will have saved to a single HDF5 file in the current working directory. 
```
my_model.h5
```

**For most models, it doesn't really make a difference which format we use but in general, using the native TensorFlow format is recommended.** 

***

We can save the model weights manually; that is, without using the `ModelCheckpoint` callback, using

```
model.save_weights('my_model')
```

In [7]:
del model, model_1
# Let's create a new model
model = create_model()

# Train the model just for 5 epochs
history = model.fit(
    x = train_images, y = train_labels,
    batch_size = 64, epochs = 5, verbose = 0,
    validation_data = (test_images, test_labels)
)

# Save the weights
model.save_weights(filepath = 'my_model.h5',
                   save_format = 'h5')


Let's check original model's loss and with loaded wieghts loss.

In [8]:
loss, acc = model.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Original Model is {:.4f}.'.format(loss))

# Build a new model
model_1 = create_model()
model_1.load_weights('my_model.h5')
loss, acc = model_1.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Loaded Weights is {:.4f}.'.format(loss))

Test Loss with Original Model is 0.4523.
Test Loss with Loaded Weights is 0.4523.


***
## Saving the entire model


If we want the entire model to be saved not just weights, then we will set the parameter `save_weights_only` to be `False`. In fact, this is the default value for this keyword argument, so we could have just left it out altogether. 

***





Whenever we save the entire model, files like the one shown below, are created.
1. my_model/assets/
2. my_model/saved_model.pb
3. my_model/variables/variables.data-000000-of-00001
4. my_model/variables/variables.index

Remember before, the callback used the file path `my_model` in the name of the files that were saved in the current working directory. Now, because we're saving the entire model and not just the weights, the file path `my_model` is used to create a subdirectory within the current working directory.

Inside that, you'll find two more subdirectories, `assets` and `variables`. The `assets` folder is where files can be stored that are used by the TensorFlow graph. In our simple example, this folder would be empty.

The `variables` folder contains the saved weights of the model. These file names should look familiar from when we were saving weights only.

The last file here, `saved_model.pb`, is the file that stores the TensorFlow graph itself. You can think of this as storing the model architecture. So it contains all the information from when we built and compiled the model, including the optimizer state, in case we want to resume training from a saved model.
***



In [9]:
del model
!rm -r save_model
# Creating a new model
model = create_model()

# Instantiating the callback to save the model
from tensorflow.keras.callbacks import ModelCheckpoint
save_model_dir = '/content/save_model/'
save_model_callback = ModelCheckpoint(
    filepath = save_model_dir,
    monitor = 'val_loss',
    verbose = 2,
    save_weights_only = False
)

# Traing using the callback
history = model.fit(x = train_images, y = train_labels,
                    batch_size = 64, epochs = 5, verbose = False,
                    validation_data = (test_images, test_labels),
                    callbacks = [save_model_callback])

rm: cannot remove 'save_model': No such file or directory

Epoch 00001: saving model to /content/save_model/
INFO:tensorflow:Assets written to: /content/save_model/assets

Epoch 00002: saving model to /content/save_model/
INFO:tensorflow:Assets written to: /content/save_model/assets

Epoch 00003: saving model to /content/save_model/
INFO:tensorflow:Assets written to: /content/save_model/assets

Epoch 00004: saving model to /content/save_model/
INFO:tensorflow:Assets written to: /content/save_model/assets

Epoch 00005: saving model to /content/save_model/
INFO:tensorflow:Assets written to: /content/save_model/assets


In [10]:
loss, acc = model.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Original Model is {:.4f}.'.format(loss))

# Build a new model
model_1 = tf.keras.models.load_model(save_model_dir)

loss, acc = model_1.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Loaded Weights is {:.4f}.'.format(loss))

Test Loss with Original Model is 0.4392.
Test Loss with Loaded Weights is 0.4392.


***
## Saving the entire model in h5 formate

we can also save the entire model in the Keras HDF5 format. In this case, just one file will be saved, which is the **keras_model.h5** file.
```
checkpoint = ModelCheckpoint('my_model.h5', save_weights_only=False) 
model.fit(x_train, y_train, epochs=10, callbacks = [checkpoint])
```

This looks the same as when we saved weights only, but this HDF5 file now contains the architecture as well as the weights.
***


We can also manually save the entire model, just as we manually saved the weights of a model after a training run. If you remember, when we saved the weights, we used `model.save_weights`. The only difference here is that we're using `model.save`. This method will now save the entire model, architecture and weights, in the same directory structure as before for the native TensorFlow saved model format.
```
model.save('my_model') # SaveModel Format
```
***


In [11]:
# Creating a new model
model2 = create_model()

# Instantiating the callback to save the model
from tensorflow.keras.callbacks import ModelCheckpoint
save_model_dir = '/content/save_model.h5'
save_model_callback = ModelCheckpoint(
    filepath = save_model_dir,
    monitor = 'val_loss',
    verbose = 2,
    save_weights_only = False
)

# Traing using the callback
history = model2.fit(x = train_images, y = train_labels,
                    batch_size = 64, epochs = 5, verbose = False,
                    validation_data = (test_images, test_labels),
                    callbacks = [save_model_callback])


Epoch 00001: saving model to /content/save_model.h5

Epoch 00002: saving model to /content/save_model.h5

Epoch 00003: saving model to /content/save_model.h5

Epoch 00004: saving model to /content/save_model.h5

Epoch 00005: saving model to /content/save_model.h5


In [12]:
loss, acc = model2.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Original Model is {:.4f}.'.format(loss))

# Build a new model
model_2 = tf.keras.models.load_model(save_model_dir)

loss, acc = model_2.evaluate(test_images, test_labels, verbose = 0)
print('Test Loss with Loaded Weights is {:.4f}.'.format(loss))

Test Loss with Original Model is 0.4432.
Test Loss with Loaded Weights is 0.4432.


***
***
<a id="part2b"></a>
## Saving the entire model

#### Create checkpoint that saves whole model, not just weights

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [14]:
# Create Tensorflow checkpoint object

checkpoint_path = 'model_checkpoints'

checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=False, 
                             save_freq='epoch', 
                             verbose=1)


#### Clear directory

In [15]:
! rm checkpoint
! rm -r save_model
! rm my_model.h5
!rm save_model.h5